#### Imports

In [1631]:
import pandas as pd
import numpy as np
import re

#### Read linkedin csv

In [1632]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
360,3770041073,Data Engineer II - NBC Sports Next,NBC Sports Next,"· Mineápolis, MN","28,26 $/h - 49,52 $/h",En remoto,Jornada completa,Intermedio,"De 1.001 a 5.000 empleados · Tecnología, infor..."
143,3758907164,Staff Machine Learning Researcher,Storm3,· San Francisco y alrededores,260.000 $/año - 300.000 $/año,Híbrido,Jornada completa,Director,De 51 a 200 empleados · Dotación y selección d...
547,3776052372,Data Engineer (Hybrid) - 16010,Enlighten,"· Columbia, MD",198.200 $/año - 297.300 $/año,En remoto,Jornada completa,Sin experiencia,De 51 a 200 empleados · Desarrollo de software


#### Cleaning 'company_state' column

In [1633]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
          'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NJ', 'NY', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT',
          'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY']
linkedin['company_state'] = linkedin['company_state'].apply(lambda row: 'Estados Unidos' if row == '· Estados Unidos' else row)
linkedin['company_state'].value_counts()

company_state
Estados Unidos         72
· Nueva York, NY       29
· Seattle, WA          19
· San Francisco, CA    17
· San José, CA         12
                       ..
· Plano, TX             1
· Lexington, KY         1
· Montpelier, VT        1
· Bolingbrook, IL       1
· Scott AFB, IL         1
Name: count, Length: 227, dtype: int64

In [1634]:
def fclean_company_state(df, states):
    for i in states:
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: i if i in str(row) else row)
    return df

linkedin = fclean_company_state(linkedin, states)

In [1635]:
def sclean_company_state(df, states):
    for key, value in states.items():
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: value if key in str(row) else row)
    return df

left_states = {
    'Virginia': 'VA',
    'Minnesota': 'MN',
    'Washington': 'WA',
    'San Francisco': 'CA',
    'Nebraska': 'NE',
    'California': 'CA',
    'Texas': 'TX',
    'Nueva York': 'NY',
    'Michigan': 'MI',
    'Luisiana': 'LA',
    'Florida': 'FL',
    'Maryland': 'MD',
    'Nevada': 'NV',
    'Oregón': 'OR',
    'Oklahoma': 'OK',
    'Georgia': 'GA',
    'Carolina del Sur': 'SC',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Arkansas': 'AR',
    'Iowa': 'IA',
    'Carolina del Norte': 'NC',
    'Indiana': 'IN',
    'Nashville': 'TN',
    'Massachusetts': 'MA',
    'Tennessee': 'TN',
    'Enid': 'OK',
    'Chicago': 'IL',
    'América del Norte': 'Estados Unidos',
    'Des Moines': 'IA',
    'Pittsburgh': 'PA',
    'Cincinnati': 'OH',
    'St. Louis': 'MO'
}
linkedin = sclean_company_state(linkedin, left_states)
linkedin['company_state'].unique()

array(['NC', 'VA', 'Estados Unidos', 'CA', 'MI', 'WA', 'NE', 'TX', 'MN',
       'MA', 'NY', 'IL', 'OH', 'GA', 'FL', 'WI', 'PA', 'OR', 'NJ', 'MD',
       'AZ', 'MO', 'KS', 'CT', 'UT', '· Unión Europea', 'IN', 'TN', 'KY',
       'LA', 'NV', 'OK', 'CO', 'IA', 'VT', 'ID', 'SC', 'HI', 'DE', 'AR',
       'RI', 'NM', 'NH'], dtype=object)

#### Cleaning 'remote_ratio' column

In [1636]:
def clean_remote_ratio(df, dict_):
    for key, val in dict_.items():
        df['remote_ratio'] = df['remote_ratio'].apply(lambda row: key if key in str(row) else str(row))
    df['remote_ratio'] = df['remote_ratio'].apply(lambda row: np.nan if str(row) == 'nan' else str(row))
    return df

remote_dict = {
    'Presencial': 0,
    'Híbrido': 50,
    'En remoto': 100
}
linkedin = clean_remote_ratio(linkedin, remote_dict)
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     261
Híbrido       185
Presencial    144
Name: count, dtype: int64

In [1637]:
linkedin.loc[linkedin['remote_ratio'] == 'nan', 'remote_ratio'] = np.nan
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     261
Híbrido       185
Presencial    144
Name: count, dtype: int64

#### Cleaning 'experience_level' column

In [1638]:
linkedin.sample(5)
linkedin['experience_level'].value_counts()

experience_level
Intermedio                 286
Sin experiencia            145
Prácticas                   66
Algo de responsabilidad     65
Director                    27
Ejecutivo                    2
Name: count, dtype: int64

#### Cleaning 'employment_type' column

In [1639]:
def clean_employment_type(df, list_):
    for i in list_:
        df['employment_type'] = df['employment_type'].apply(lambda row: i if i in str(row) else str(row))
    return df

remote_dict = [
    'Jornada completa',
    'Contrato por obra',
    'Media jornada',
    'Prácticas'
]
linkedin = clean_employment_type(linkedin, remote_dict)
linkedin['employment_type'].value_counts()

employment_type
Jornada completa     418
Contrato por obra     88
Prácticas             52
Media jornada         38
Name: count, dtype: int64

#### Cleaning 'company_size' column

In [1640]:
linkedin['company_size'].value_counts()

company_size
De 51 a 200 empleados · Servicios y consultoría de TI                                               37
De 51 a 200 empleados · Dotación y selección de personal                                            32
De 11 a 50 empleados · Dotación y selección de personal                                             25
Más de 10.001 empleados · Hospitales y atención sanitaria                                           24
De 1.001 a 5.000 empleados · Servicios y consultoría de TI                                          23
                                                                                                    ..
Ve una comparación con los otros 301 solicitantes. Probar Premium gratis                             1
De 1.001 a 5.000 empleados · Fabricación de alimentos y bebidas                                      1
Más de 10.001 empleados · Fabricación de electrodomésticos y productos eléctricos y electrónicos     1
Entre 1 y 10 empleados · Construcción                       

In [1641]:
linkedin['employees'] = linkedin['company_size'].apply(lambda row: row.split('·')[0].strip())
linkedin['employees'].value_counts()

employees
Más de 10.001 empleados                                                     126
De 51 a 200 empleados                                                       119
De 1.001 a 5.000 empleados                                                  100
De 11 a 50 empleados                                                         68
De 201 a 500 empleados                                                       52
De 501 a 1.000 empleados                                                     51
De 5.001 a 10.000 empleados                                                  40
Entre 1 y 10 empleados                                                       33
Nagesh Kumar Gona ✈ busca personal para este empleo                           1
Ve una comparación con los otros 137 solicitantes. Probar Premium gratis      1
Ve una comparación con los otros 301 solicitantes. Probar Premium gratis      1
Ve una comparación con los otros 652 solicitantes. Probar Premium gratis      1
Fabricación de productos de pl

In [1642]:
# Patrones que deseas buscar en la columna específica
patrones = [
    r"Ve una comparación con los otros \d+ solicitantes\. Probar Premium gratis",
    r"Fabricación de productos de plástico",
    r"Servicios para el individuo y la familia"
]

# Crear una expresión regular combinando los patrones con el operador OR (|)
regex_pattern = '|'.join(patrones)

# Filtrar el DataFrame por la columna específica
linkedin[linkedin['employees'].str.contains(regex_pattern)]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,employees
114,3756057160,Lead Machine Learning Engineer,Non-Disclosed,NY,850.000 $/año - 1.100.000 $/año,Presencial,Jornada completa,Director,Ve una comparación con los otros 137 solicitan...,Ve una comparación con los otros 137 solicitan...
194,3762242381,Data Scientist,Probar Premium gratis,IL,170.000 $/año - 190.000 $/año,Híbrido,Jornada completa,Intermedio,Ve una comparación con los otros 301 solicitan...,Ve una comparación con los otros 301 solicitan...
195,3762242406,Data Analyst,Probar Premium gratis,IL,170.000 $/año - 190.000 $/año,Híbrido,Jornada completa,Intermedio,Ve una comparación con los otros 652 solicitan...,Ve una comparación con los otros 652 solicitan...
409,3771514066,Quality Control Analyst,"Carolina Poly, Inc.",SC,65.000 $/año - 72.000 $/año,Presencial,Jornada completa,Sin experiencia,Fabricación de productos de plástico,Fabricación de productos de plástico
504,3775200323,Data Analyst,Probar Premium gratis,MA,35 $/h - 40 $/h,En remoto,Media jornada,NaN,Ve una comparación con los otros 444 solicitan...,Ve una comparación con los otros 444 solicitan...
528,3775649749,Data Consultant ( Remote),Fresca Fresh Blanket Service,Estados Unidos,NaN,En remoto,Media jornada,Algo de responsabilidad,Servicios para el individuo y la familia,Servicios para el individuo y la familia


In [1643]:
linkedin.loc[linkedin['employees'].str.contains(regex_pattern), 'employees'] = np.nan
linkedin['employees'].value_counts()

employees
Más de 10.001 empleados                                126
De 51 a 200 empleados                                  119
De 1.001 a 5.000 empleados                             100
De 11 a 50 empleados                                    68
De 201 a 500 empleados                                  52
De 501 a 1.000 empleados                                51
De 5.001 a 10.000 empleados                             40
Entre 1 y 10 empleados                                  33
Nagesh Kumar Gona ✈ busca personal para este empleo      1
Name: count, dtype: int64

In [1644]:
pattern1 = r"Más de (\d+\.?\d*) empleados"
pattern2 = r"De (\d+\.?\d*) a (\d+\.?\d*) empleados"
pattern3 = r"Entre (\d+\.?\d*) y (\d+\.?\d*) empleados"
def clean_employee(row):
    match1 = re.match(pattern1, str(row))
    match2 = re.match(pattern2, str(row))
    match3 = re.match(pattern3, str(row))
    if match1:
        return match1.group(1)
    elif match2:
        return match2.group(2)
    elif match3:
        return match3.group(2)
    else:
        return np.nan

linkedin['employees'] = linkedin['employees'].apply(clean_employee)
linkedin['employees'].value_counts()

employees
10.001    126
200       119
5.000     100
50         68
500        52
1.000      51
10.000     40
10         33
Name: count, dtype: int64

In [1645]:
def standarize_company_size(row):
    if pd.notnull(row):
        if int(row.replace('.', '')) < 50:
            return 'S'
        elif int(row.replace('.', '')) <= 250:
            return 'M'
        elif int(row.replace('.', '')) > 250:
            return 'L'
        else:
            return np.nan

linkedin['company_size'] = linkedin['employees'].apply(standarize_company_size)
linkedin['company_size'].value_counts()

company_size
L    369
M    187
S     33
Name: count, dtype: int64

#### Cleaning 'salary_range' column

In [1646]:
linkedin['salary_range'].unique()

array(['100.000 $/año - 150.000 $/año', '283.780 $/año - 331.640 $/año',
       '145.000 $/año - 180.000 $/año', '95.000 $/año - 130.000 $/año',
       '167.200 $/año - 250.800 $/año', '200.000 $/año - 240.000 $/año',
       '70.000 $/yr', nan, '200.000 $/año - 300.000 $/año',
       '120.000 $/año - 220.000 $/año', '144.000 $/año - 270.250 $/año',
       '170.112 $/año - 237.000 $/año', '170.000 $/año - 190.000 $/año',
       '20 $/hr', '145.000 $/año - 195.000 $/año',
       '67.733 $/año - 91.440 $/año', '95.000 $/año - 120.000 $/año',
       '78 $/h - 104 $/h', '122.000 $/año - 190.000 $/año',
       '101.000 $/año - 179.000 $/año', '48 $/h - 65 $/h',
       '125.000 $/año - 160.000 $/año', '850.000 $/año - 1.100.000 $/año',
       '75 $/h - 85 $/h', '45 $/h - 55 $/h',
       '150.000 $/año - 250.000 $/año', '40 $/h - 43 $/h',
       '150.000 $/año - 300.000 $/año', '90 $/hr',
       '260.000 $/año - 300.000 $/año', '150.000 $/año - 200.000 $/año',
       '90.000 $/año - 130.000 $/

In [1647]:
hour_pattern = r"(\d+) \$\/hr"
year_pattern = r"(\d+\.?\d*) \$\/yr"
hour_range_pattern = r"(\d+\,?\d*) \$\/h - (\d+\,?\d*) \$\/h"
year_range_pattern = r"(\d+\.?\d*) \$\/año - (\d+\.?\d*) \$\/año"
month_range_pattern = r"(\d+\.?\d*) \$\/mes - (\d+\.?\d*) \$\/mes"


def clean_salary(row):
    match_h = re.match(hour_pattern, str(row))
    match_y = re.match(year_pattern, str(row))
    match_rh = re.match(hour_range_pattern, str(row))
    match_ry = re.match(year_range_pattern, str(row))
    match_rm = re.match(month_range_pattern, str(row))
    if match_h:
        num_h = match_h.group(1)
        num_w = int(num_h) * 40
        return int(num_w * 48)
    elif match_y:
        return int(match_y.group(1).replace('.', ''))
    elif match_rh:
        min_ = match_rh.group(1)
        max_ = match_rh.group(2)
        num_h = float(min_.replace(',', '.')) + float(max_.replace(',', '.')) / 2
        num_w = num_h * 40
        return int(num_w * 48)
    elif match_ry:
        min_ = match_ry.group(1)
        max_ = match_ry.group(2)
        if len(min_) <= 3:
            min_ += '000'
        if len(max_) <= 3:
            max_ += '000'
        num_y = float(min_.replace('.', '')) + float(max_.replace('.', '')) / 2
        return int(num_y)
    elif match_rm:
        min_ = match_rm.group(1)
        max_ = match_rm.group(2)
        num_m = float(min_.replace('.', '')) + float(max_.replace('.', '')) / 2
        return int(num_m * 12)
    else:
        return np.nan

linkedin['salary_range'] = linkedin['salary_range'].apply(clean_salary)
linkedin['salary_range'].value_counts()

salary_range
230000.0    23
207750.0    21
250000.0    19
279125.0    19
57600.0     18
            ..
148800.0     1
16205.0      1
192000.0     1
157440.0     1
217500.0     1
Name: count, Length: 113, dtype: int64

In [1648]:
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,employees
393,3770786405,"Staff Credit Strategy Analyst, Data Scientist",SoFi,DE,325000.0,Híbrido,Jornada completa,Intermedio,L,5.000
128,3757366936,Applied Scientist,Quanti Recruitment,NY,275000.0,Presencial,Jornada completa,Intermedio,S,10
231,3764675617,Statistician/Data Scientist,Eli Lilly and Company,IN,96960.0,Presencial,Contrato por obra,Intermedio,L,10.001
102,3754419732,Data Scientist MMM (Marketing Mix Model),AMISEQ,CA,NaN,Híbrido,Prácticas,Prácticas,M,200
408,3771503379,Data Analyst,"Business Plan Solutions, LLC",TX,101000.0,Presencial,Jornada completa,Sin experiencia,M,50


#### Cleaning 'job_title' column

In [1649]:
linkedin['job_title'].value_counts()

job_title
Machine Learning Engineer                           59
Data Engineer                                       44
Data Analyst                                        43
Data Scientist                                      35
Data Analyst  I                                     19
                                                    ..
Data Scientist - LLM                                 1
Actuarial Data Scientist                             1
AI/ML Data Scientist                                 1
Associate Data Scientist - Cybersecurity, Intern     1
Machine Learning Engineer, Planning                  1
Name: count, Length: 325, dtype: int64

In [1650]:
def standarize_job(row):
    title = str(row).lower()
    if 'data engineer' in title:
        return 'Data Engineer'
    elif 'data analyst' in title:
        return 'Data Analyst'
    elif 'data scientist' in title:
        return 'Data Scientist'
    elif 'data' in title and 'junior' not in title:
        if 'engineer' in title:
            return 'Data Engineer'
        elif 'analyst' in title:
            return 'Data Analyst'
    elif ('machine' in title and 'learning' in title) or ('ml' in title and 'engineer' in title):
        return 'Machine Learning Engineer'
    elif 'deep' in title and 'learning' in title and 'engineer' in title:
        return 'Deep Learning Engineer'
    elif 'junior' in title:
        if 'data' in title and 'engineer' in title:
            return 'Data Engineer'
        elif 'data' in title and 'analyst' in title:
            return 'Data Analyst'
    elif 'business' in title:
        if 'intelligence' in title and 'analyst' in title:
            return 'BI Analyst'
        elif 'analyst' in title:
            return 'Business Analyst'
        else:
            return 'BI Engineer'
    elif 'artificial intelligence' in title or 'ai' in title:
        if 'research engineer' in title:
            return 'Research Engineer'
        elif 'ml' in title:
            return 'Machine Learning Engineer'
        elif 'engineer' in title:
            return 'AI Engineer'
        else:
            return title
    else:
        return str(row)


linkedin['job_title'] = linkedin['job_title'].apply(standarize_job)
linkedin['job_title'].unique()

array(['Machine Learning Engineer', 'CYBER INTEL ANALYST - PEN TESTER',
       'Deep Learning Engineer',
       'Senior Algorithms Engineer (Image Processing)', 'Data Engineer',
       'Data Scientist',
       'Computer Vision Engineer (Multimodal with Large Language Models)',
       'BI Analyst', 'Data Analyst', 'Artificial Learning Engineer',
       'Research Engineer', 'Computer Vision Engineer',
       'RF Computational Engineer 3', 'Project Scheduler',
       'Adaptive Optics Senior Software Engineer',
       'Finance Analyst (Benelux)',
       'Core Infrastructure Engineer - Networking', 'Analytics Engineer',
       'Applied Scientist', None, 'Quantitative Researcher',
       'Computer Vision and Optimization Engineer', 'AI Engineer',
       'Software Company DevOps Engineer',
       'summer intern - r&d/artificial intelligence',
       'Remote Cruise Planner - Entry Level',
       'Remote Travel Consultant - Entry Level',
       'Remote Customer Service Sales Rep - Entry Level',

    Categories for 'job_title' column

In [1651]:
categories = [
    'Data Engineer',
    'Data Analyst',
    'Data Scientist',
    'Machine Learning Engineer',
    'Deep Learning Engineer',
    'BI Analyst',
    'Business Analyst',
    'BI Engineer',
    'Research Engineer',
    'AI Engineer'
]

In [1652]:
linkedin.shape

(596, 10)

In [1653]:
linkedin = linkedin[linkedin['job_title'].isin(categories)]
linkedin = linkedin.drop(columns=['employees'])

In [1654]:
linkedin.shape

(518, 9)

In [1656]:
linkedin[linkedin['job_id'] == 3756135992]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
115,3756135992,Data Scientist,Adobe,CA,NaN,Presencial,Jornada completa,Director,L


In [1655]:
linkedin.to_csv('../data/linkedin_standarized.csv', index=False)